<a href="https://colab.research.google.com/github/nnilayy/Unet/blob/main/Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt 
import tensorflow as tf 
import numpy as np
import cv2 
import os 

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!mkdir "/content/drive/MyDrive/dataset"

In [ ]:
!unzip -q /content/drive/MyDrive/dataset.zip -d /content/drive/MyDrive/

In [12]:
train=ImageDataGenerator(rescale=1/255,
                         rotation_range=40,
                         brightness_range=(0,5),
                         zoom_range=0.2,
                         shear_range=0.2,
                         horizontal_flip=True,
                         vertical_flip=True,
                         fill_mode="nearest",
                         )

valid=ImageDataGenerator(rescale=1/255,
                              rotation_range=40,
                              brightness_range=(0,5),
                              zoom_range=0.2,
                              shear_range=0.2,
                              horizontal_flip=True,
                              vertical_flip=True,
                              fill_mode="nearest",)

In [16]:
train_dataset=train.flow_from_directory('/content/drive/MyDrive/dataset_3/train',
                                        target_size=(128,128),
                                        batch_size=5,
                                        class_mode='binary',
                                        color_mode='grayscale',
                                        )
val_dataset=valid.flow_from_directory('/content/drive/MyDrive/dataset_3/validation',
                                        target_size=(128,128),
                                        batch_size=5,
                                        class_mode='binary',
                                        color_mode='grayscale'
                                        )

Found 301 images belonging to 2 classes.
Found 90 images belonging to 2 classes.


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, AveragePooling2D, Flatten, Dropout, BatchNormalization, InputLayer, Reshape

model = Sequential()
model.add(InputLayer(input_shape=(128, 128,1)))
model.add(Conv2D(8, 3, padding='same', activation="relu"))
model.add(MaxPooling2D(2, padding='same'))
model.add(Conv2D(16, 3, padding='same', activation="relu"))
model.add(MaxPooling2D(2, padding='same'))
model.add(Conv2D(8, 3, padding='same', activation="relu"))
model.add(MaxPooling2D(2, padding='same'))
model.add(Conv2D(4, 3, padding='same', activation="relu"))
model.add(MaxPooling2D(2, padding='same'))
model.add(Conv2D(4, 3, padding='same', activation="relu"))
model.add(MaxPooling2D(2, padding='same'))
model.add(MaxPooling2D(2, padding='same'))
model.add(Flatten())
model.add(Dense(1, activation="sigmoid"))
model.summary()






# ---------------------------------------------------------------------------------------------------
# ---------------------------------------------------------------------------------------------------
# (1)Params: 32,000
# (2)Train_Accuracy: 85.45
# (3)Val_Accuracy: 84.44
# model = Sequential()
# model.add(InputLayer(input_shape=(224, 224,3)))
# model.add(Conv2D(16, 3, padding='same', activation="relu"))
# model.add(MaxPooling2D(2, padding='same'))
# model.add(Conv2D(16, 3, padding='same', activation="relu"))
# model.add(MaxPooling2D(2, padding='same'))
# model.add(Conv2D(16, 3, padding='same', activation="relu"))
# model.add(MaxPooling2D(2, padding='same'))
# model.add(MaxPooling2D(2, padding='same'))
# model.add(MaxPooling2D(2, padding='same'))
# model.add(Flatten())
# model.add(Dense(32, activation="relu"))
# model.add(Dense(1, activation="sigmoid"))

In [ ]:
for layer in model.layers:
  print(layer.get_config()['name'])

In [20]:
model.compile(loss="binary_crossentropy", 
              optimizer=tf.keras.optimizers.RMSprop(learning_rate=1e-3), 
              metrics=["accuracy"])

history=model.fit(train_dataset, 
                  steps_per_epoch=16, 
                  epochs=100, 
                  validation_data=val_dataset)

Epoch 1/100
16/16 [==============================] - 3s 82ms/step - loss: 0.6958 - accuracy: 0.4125 - val_loss: 0.6915 - val_accuracy: 0.5667
Epoch 2/100
16/16 [==============================] - 1s 83ms/step - loss: 0.6948 - accuracy: 0.4875 - val_loss: 0.6927 - val_accuracy: 0.5333
Epoch 3/100
16/16 [==============================] - 1s 82ms/step - loss: 0.6931 - accuracy: 0.5375 - val_loss: 0.6967 - val_accuracy: 0.4222
Epoch 4/100
16/16 [==============================] - 1s 59ms/step - loss: 0.6954 - accuracy: 0.4750 - val_loss: 0.6930 - val_accuracy: 0.5556
Epoch 5/100
16/16 [==============================] - 1s 60ms/step - loss: 0.6929 - accuracy: 0.5375 - val_loss: 0.6920 - val_accuracy: 0.5667
Epoch 6/100
16/16 [==============================] - 1s 58ms/step - loss: 0.6904 - accuracy: 0.5658 - val_loss: 0.6885 - val_accuracy: 0.5556
Epoch 7/100
16/16 [==============================] - 1s 58ms/step - loss: 0.6792 - accuracy: 0.6250 - val_loss: 0.6862 - val_accuracy: 0.5556
Epoch 

In [21]:
model.save("/content/drive/MyDrive/model_128.h5")

In [ ]:
file="/content/images.jpg"
image=cv2.imread(file)
image=cv2.resize(image,(224,224))
image=np.expand_dims(image,0)
prediction=int(model.predict(image))

if prediction == 0:
  print('Cracked')

elif prediction==1:
  print('Not Cracked')

In [ ]:
# https://www.youtube.com/watch?v=uqomO_BZ44g

In [22]:
import os
def get_file_size(file_path):
  size=os.path.getsize(file_path)
  return size

In [23]:
def convert_bytes(size,unit=None):
  if unit=="KB":
    return print("File size: "+str(round(size/1024,3))+" Kilobytes")
  elif unit=="MB":
    return print("File size: "+str(round(size/(1024*1024),3))+" Megabytes")
  else:
    return print("File size: "+str(size)+"bytes")

In [26]:
model="/content/drive/MyDrive/model_128.h5"

In [27]:
# Model Size Before Model Quantization
convert_bytes(get_file_size(model),"MB")

File size: 0.07 Megabytes


In [ ]:
import tensorflow as tf
model=tf.keras.models.load_model(model)
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations=[tf.lite.Optimize.DEFAULT]
converter.experimental_new_converter = True
tflite_model_name="model.tflite"
tflite_model = converter.convert()
open(tflite_model_name,"wb").write(tflite_model)

In [29]:
# Model Size After Model Quantization
tflite_model="/content/model.tflite"
convert_bytes(get_file_size(tflite_model),"KB")

File size: 10.031 Kilobytes


In [ ]:
# ()Dataset 
# ()Inferencing Code on Arduino
# ()SD Card model transfer
# ()Library Creation and function deletion

In [30]:
import binascii
def convert_to_c_array(bytes) -> str:
  hexstr = binascii.hexlify(bytes).decode("UTF-8")
  hexstr = hexstr.upper()
  array = ["0x" + hexstr[i:i + 2] for i in range(0, len(hexstr), 2)]
  array = [array[i:i+10] for i in range(0, len(array), 10)]
  return ",\n  ".join([", ".join(e) for e in array])

In [31]:
tflite_binary = open('model.tflite', 'rb').read()
ascii_bytes = convert_to_c_array(tflite_binary)
header_file = "const unsigned char model_tflite[] = {\n  " + ascii_bytes + "\n};\nunsigned int model_tflite_len = " + str(len(tflite_binary)) + ";"
open("model_128.h", "w").write(header_file)
convert_bytes(get_file_size('/content/model_128.h'),"KB")

File size: 62.272 Kilobytes


In [ ]:
# train_data=train_dataset[0]
# X_train=train_data[0]
# y_train=train_data[1]

# test_data=val_dataset[0]
# X_test=test_data[0]
# y_test=test_data[1]

# print(X_train.shape,X_test.shape,y_train.shape,y_test.shape)
# # print(y_test)

(301, 130, 130, 1) (90, 130, 130, 1) (301,) (90,)
